In [1]:
from datasets import load_dataset
from transformers import pipeline

In [2]:
ds = load_dataset("Yelp/yelp_review_full")
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
pipe = pipeline(
    'text-classification',
    model='tabularisai/multilingual-sentiment-analysis',
    padding=True,
    truncation=True,
    max_length=512
)

Device set to use cuda:0


In [4]:
label_to_star = {
    'Very Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Very Positive': 4
}

def predict(batch):
    results = pipe(batch['text'])
    batch['pred'] = [r['label'] for r in results]
    return batch

test_dataset = ds['test']
test_dataset = test_dataset.map(predict, batched=True, batch_size=512)

correct, polarity_correct = 0, 0
for sample in test_dataset:
    star_gt = sample['label']
    star_pred = label_to_star[sample['pred']]

    if star_gt == star_pred:
        correct += 1
    
    if (star_gt > 2 and star_pred > 2) or (star_gt <= 2 and star_pred <= 2):
        polarity_correct += 1

accuracy = correct / len(test_dataset)
polarity_accuracy = polarity_correct / len(test_dataset)

print(f'Accuracy: {accuracy}')
print(f'Polarity Accuracy: {polarity_accuracy}')

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Accuracy: 0.44192
Polarity Accuracy: 0.81978
